# Notebook 1: Extract Z-scored LFP

- Notebook for extracting local field potential traces from Spikegadgets `.rec` files

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [48]:
ORIGINAL_TRIAL_COL = "condition"
TRIAL_OUTCOME_COL = "trial_outcome"

In [49]:
# NOTE: Change based on individual project data location

# Spreadsheet of channel mapping
CHANNEL_MAPPING_DF = pd.read_excel("../../data/channel_mapping.xlsx")
# Spreadsheet of tone time
TONE_TIMESTAMP_DF = pd.read_excel("../../data/rce_tone_timestamp.xlsx", index_col=0)

In [50]:
CHANNEL_MAPPING_DF.head()

,Cohort,Subject,eib_mPFC,eib_vHPC,eib_BLA,eib_LH,eib_MD,spike_interface_mPFC,spike_interface_vHPC,spike_interface_BLA,spike_interface_LH,spike_interface_MD
0,1,6.1,NaN,15,14,13,31,21.0,15.0,14.0,13.0,16.0
1,1,6.2,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
2,1,6.3,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
3,1,6.4,NaN,15,14,13,31,NaN,NaN,NaN,NaN,NaN
4,2,1.1,NaN,16,17,18,19,5.0,31.0,30.0,29.0,28.0


In [51]:
TONE_TIMESTAMP_DF.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,condition,competition_closeness,Unnamed: 13
2299.0,3772337.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,0.0,20221122_161341_omission_subject_6_1_and_6_3.1...,0.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2300.0,5204112.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,1431775.0,20221122_161341_omission_subject_6_1_and_6_3.1...,1784.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2301.0,6804107.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,3031770.0,20221122_161341_omission_subject_6_1_and_6_3.1...,3779.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2302.0,8604101.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,4831764.0,20221122_161341_omission_subject_6_1_and_6_3.1...,6021.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2303.0,10204096.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,6431759.0,20221122_161341_omission_subject_6_1_and_6_3.1...,8015.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN


In [52]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/*/*.rec")

In [53]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115641_standard_comp_to_omission_D1_subj_2-2_and_2-4.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_02/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_03/20221203_154800_omission_and_competition_subject_6_4_and_6_1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100646_standard_comp_to_omission_D2_subj_2-4_and_2-1.rec',
 '/scratch/back

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [54]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"../../proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [55]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Reformatting Dataframe

- Dropping all rows that have not been labeled

In [56]:
all_trials_df = TONE_TIMESTAMP_DF.dropna(subset=ORIGINAL_TRIAL_COL).reset_index(drop=True)

In [17]:
sorted(all_trials_df["recording_dir"].unique())

['20221202_134600_omission_and_competition_subject_6_1_and_6_2',
 '20221203_154800_omission_and_competition_subject_6_4_and_6_1',
 '20221214_125409_om_and_comp_6_1_and_6_3',
 '20221215_145401_comp_amd_om_6_1_and_6_3',
 '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3',
 '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
 '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
 '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
 '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
 '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2']

- Making the video frame number usable

In [18]:
all_trials_df["video_frame"] = all_trials_df["video_frame"].astype(int)

- Getting the name of the video so that we can sync it up with the ephys recording

In [19]:
all_trials_df["video_name"]  = all_trials_df["video_file"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

- Getting all subject IDs for a given recording

In [20]:
# using different id extractions for different file formats
all_trials_df["all_subjects"] = all_trials_df["recording_dir"].apply(lambda x: x if "2023" in x else "subj" + "_".join(x.split("_")[-5:]))
all_trials_df["all_subjects"] = all_trials_df["all_subjects"].apply(lambda x: tuple(sorted([num.strip("_").replace("_",".") for num in x.replace("-", "_").split("subj")[-1].strip("_").split("and")])))

In [21]:
all_trials_df["all_subjects"].unique()

array([('6.1', '6.2'), ('6.1', '6.4'), ('6.1', '6.3'), ('1.3', '1.4'),
       ('1.1', '1.2'), ('1.1', '1.4'), ('1.2', '1.4')], dtype=object)

In [22]:
all_trials_df["current_subject"] = all_trials_df["subject_info"].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2])).astype(str)

In [23]:
all_trials_df["current_subject"].unique()

array(['6.1', '1.3', '1.4', '1.1', '1.2'], dtype=object)

- Converting the trial label to win or lose based on who won the trial

In [24]:
all_trials_df[TRIAL_OUTCOME_COL] = all_trials_df.apply(
    lambda x: "win" if str(x[ORIGINAL_TRIAL_COL]).strip() == str(x["current_subject"]) 
             else ("lose" if str(x[ORIGINAL_TRIAL_COL]) in x["all_subjects"] 
                   else x[ORIGINAL_TRIAL_COL]), axis=1)

In [25]:
all_trials_df[TRIAL_OUTCOME_COL].unique()

array(['rewarded', 'omission', 'win', 'lose'], dtype=object)

- Adding the competition closeness as a column

In [26]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in all_trials_df["competition_closeness"].unique()}

In [27]:
competition_closeness_map

{nan: nan,
 'Subj 1 Only': 'non_comp',
 'Subj 2 blocking Subj 1': 'comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 2 Only': 'non_comp',
 'Subj 2 then Subj 1': 'comp',
 'Close Call': 'comp'}

In [28]:
all_trials_df["competition_closeness"] = all_trials_df["competition_closeness"].map(competition_closeness_map)

In [29]:
all_trials_df["competition_closeness"] = all_trials_df.apply(lambda x: "_".join([str(x[TRIAL_OUTCOME_COL]), str(x["competition_closeness"])]).strip("nan").strip("_"), axis=1)

In [30]:
all_trials_df["competition_closeness"].unique()

array(['rewarded', 'omission', 'win_non_comp', 'win_comp',
       'lose_non_comp', 'lose_comp'], dtype=object)

- Adding the LFP index

In [31]:
all_trials_df["lfp_index"] = (all_trials_df["time_stamp_index"] // (EPHYS_SAMPLING_RATE/LFP_SAMPLING_RATE)).astype(int)

In [32]:
all_trials_df["time"] = all_trials_df["time"].astype(int)

In [33]:
all_trials_df["time_stamp_index"] = all_trials_df["time_stamp_index"].astype(int)

- Removing unnecessary columns

In [34]:
all_trials_df = all_trials_df.drop(columns=["state", "din", ORIGINAL_TRIAL_COL, "Unnamed: 13"], errors="ignore")

In [35]:
all_trials_df.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,video_name,all_subjects,current_subject,trial_outcome,lfp_index
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,"(6.1, 6.2)",6.1,rewarded,69541
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,"(6.1, 6.2)",6.1,rewarded,149541
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,"(6.1, 6.2)",6.1,rewarded,239541
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,20221202_134600_omission_and_competition_subje...,"(6.1, 6.2)",6.1,omission,319541
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,20221202_134600_omission_and_competition_subje...,"(6.1, 6.2)",6.1,rewarded,394541


In [36]:
all_trials_df.groupby(["competition_closeness"]).count()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,video_name,all_subjects,current_subject,trial_outcome,lfp_index
competition_closeness,,,,,,,,,,,,,
lose_comp,105,105,105,105,105,105,105,105,105,105,105,105,105
lose_non_comp,89,89,89,89,89,89,89,89,89,89,89,89,89
omission,48,48,48,48,48,48,48,48,48,48,48,48,48
rewarded,290,290,290,290,290,290,290,290,290,290,290,290,290
win_comp,77,77,77,77,77,77,77,77,77,77,77,77,77
win_non_comp,134,134,134,134,134,134,134,134,134,134,134,134,134


- Making columns of the different timestamps

In [37]:
all_trials_df["baseline_lfp_timestamp_range"] = all_trials_df["lfp_index"].apply(lambda x: (x - TRIAL_DURATION * LFP_SAMPLING_RATE, x))

In [38]:
all_trials_df["trial_lfp_timestamp_range"] = all_trials_df["lfp_index"].apply(lambda x: (x, x + TRIAL_DURATION * LFP_SAMPLING_RATE))

In [39]:
all_trials_df["baseline_ephys_timestamp_range"] = all_trials_df["time_stamp_index"].apply(lambda x: (x - TRIAL_DURATION * EPHYS_SAMPLING_RATE, x))

In [40]:
all_trials_df["trial_ephys_timestamp_range"] = all_trials_df["time_stamp_index"].apply(lambda x: (x, x + TRIAL_DURATION * EPHYS_SAMPLING_RATE))

In [41]:
all_trials_df["baseline_videoframe_range"] = all_trials_df["video_frame"].apply(lambda x: (x - TRIAL_DURATION * FRAME_RATE, x))

In [42]:
all_trials_df["trial_videoframe_range"] = all_trials_df["video_frame"].apply(lambda x: (x, x + TRIAL_DURATION * FRAME_RATE))

In [43]:
raise ValueError()

ValueError: 

# Extracting the LFP

In [ ]:
recording_name_to_all_ch_lfp = {}
# Going through all the recording sessions 
for session_dir in ALL_SESSION_DIR:
    # Going through all the recordings in each session
    for recording_path in glob.glob(os.path.join(session_dir, RECORDING_EXTENTION)):
        try:
            recording_basename = os.path.splitext(os.path.basename(recording_path))[0]
            # checking to see if the recording has an ECU component
            # if it doesn't, then the next one be extracted
            current_recording = se.read_spikegadgets(recording_path, stream_id=ECU_STREAM_ID)
            current_recording = se.read_spikegadgets(recording_path, stream_id=TRODES_STREAM_ID)
            print(recording_basename)
            # Preprocessing the LFP
            current_recording = sp.bandpass_filter(current_recording, freq_min=LFP_FREQ_MIN, freq_max=LFP_FREQ_MAX)
            current_recording = sp.notch_filter(current_recording, freq=ELECTRIC_NOISE_FREQ)
            current_recording = sp.resample(current_recording, resample_rate=LFP_SAMPLING_RATE)
            current_recording = sp.zscore(current_recording)
            recording_name_to_all_ch_lfp[recording_basename] = current_recording
        except Exception as error:
            # handle the exception
            print("An exception occurred:", error) # An exception occurred: division by zero




- Filtering for all trials that we got the LFP for

In [ ]:
all_trials_df = all_trials_df[all_trials_df["recording_file"].isin(recording_name_to_all_ch_lfp.keys())].reset_index(drop=True)

In [ ]:
all_trials_df.head()

- Adding trial numbers based on timestamp ordering for each recording

In [ ]:
all_trials_df = all_trials_df.groupby('recording_file').apply(lambda g: compute_sorted_index(g, value_column='time', index_column='trial_number')).reset_index(drop=True)

In [ ]:
all_trials_df["trial_number"] = all_trials_df["trial_number"] + 1

## Adding the LFP trace information

In [ ]:
CHANNEL_MAPPING_DF

- Adding all the brain region to ch information

In [ ]:
CHANNEL_MAPPING_DF["Subject"] = CHANNEL_MAPPING_DF["Subject"].astype(str)

In [ ]:
channel_map_and_all_trials_df = all_trials_df.merge(CHANNEL_MAPPING_DF, left_on="current_subject", right_on="Subject", how="left")

In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=[col for col in channel_map_and_all_trials_df.columns if "eib" in col], errors="ignore")

In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=["Subject"], errors="ignore")

In [ ]:
channel_map_and_all_trials_df.head()

In [ ]:
channel_map_and_all_trials_df.to_csv("./proc/trial_metadata.csv")

In [ ]:
channel_map_and_all_trials_df.to_pickle("./proc/trial_metadata.pkl")

In [ ]:
channel_map_and_all_trials_df.columns

- Linking up all LFP calculations with all the trials

In [ ]:
channel_map_and_all_trials_df["all_ch_lfp"] = channel_map_and_all_trials_df["recording_file"].map(recording_name_to_all_ch_lfp)

- Creating a new row for each brain region

In [ ]:
brain_region_col = [col for col in CHANNEL_MAPPING_DF if "spike_interface" in col]

In [ ]:
id_cols = [col for col in channel_map_and_all_trials_df.columns if col not in brain_region_col]

In [ ]:
brain_region_col

In [ ]:
for col in brain_region_col:
    channel_map_and_all_trials_df[col] = channel_map_and_all_trials_df[col].astype(int).astype(str)

In [ ]:
channel_map_and_all_trials_df.columns

- Extracting the traces for each trial and brain region

In [ ]:
for col in brain_region_col:
    print(col)
    brain_region = col.strip("spike_interface").strip("_")
    baseline_trace_column = "{}_baseline_lfp_trace".format(brain_region)
    trial_trace_column = "{}_trial_lfp_trace".format(brain_region)
    baselineandtrial_trace_column = "{}_baseline-trial_lfp_trace".format(brain_region)
    
    channel_map_and_all_trials_df[baseline_trace_column] = channel_map_and_all_trials_df.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["baseline_lfp_timestamp_range"][0], end_frame=row["baseline_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_all_trials_df[trial_trace_column] = channel_map_and_all_trials_df.apply(lambda row: row["all_ch_lfp"].get_traces(channel_ids=[row[col]], start_frame=row["trial_lfp_timestamp_range"][0], end_frame=row["trial_lfp_timestamp_range"][1]).T[0], axis=1)

    channel_map_and_all_trials_df[baselineandtrial_trace_column] = channel_map_and_all_trials_df.apply(lambda row: np.concatenate([row[baseline_trace_column], row[trial_trace_column]]), axis=1)                                                                                                              
                                                                                                                                                           
                                                                                                                                                    

In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=["all_ch_lfp"], errors="ignore")

In [ ]:
channel_map_and_all_trials_df = channel_map_and_all_trials_df.drop(columns=[col for col in channel_map_and_all_trials_df if "spike_interface" in col], errors="ignore")

In [ ]:
channel_map_and_all_trials_df.head()

In [ ]:
channel_map_and_all_trials_df.columns

In [ ]:
channel_map_and_all_trials_df.to_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")